In [ ]:
# Cell 1 — Setup: install packages and load env
import sys, os, subprocess, importlib.util
from pathlib import Path

def ensure_packages(packages):
    for module_name, pip_spec in packages:
        try:
            if importlib.util.find_spec(module_name) is None:
                subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pip_spec])
        except Exception as e:
            print(f'Package install skipped/failed for {pip_spec}: {e}')

BASE_PACKAGES = [
    ('openai', 'openai>=1.47.0'),
    ('dotenv', 'python-dotenv>=1.0.1'),
    ('ipywidgets', 'ipywidgets>=8.1.3'),
    ('nest_asyncio', 'nest-asyncio>=1.6.0'),
    ('tqdm', 'tqdm>=4.66.5'),
    ('httpx', 'httpx>=0.27.0'),
    ('pydantic', 'pydantic>=2.7.0'),
]

ensure_packages(BASE_PACKAGES)

# Try to install AgentKit if available on PyPI
for candidate in ('openai-agentkit', 'agentkit'):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', candidate])
        break
    except Exception:
        pass

# Enable nested asyncio in notebooks
import nest_asyncio as _nest_asyncio
_nest_asyncio.apply()

# Load environment from .env if present
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

if not os.environ.get('OPENAI_API_KEY'):
    print('Warning: set OPENAI_API_KEY in your environment or .env before running your agent.')


In [ ]:
# Cell 2 — Paste your Agent Builder code below.
#
# Guidelines:
# - Define an entrypoint callable (function or class) if possible.
# - The runner in Cell 3 will auto-detect common names like: run, main, agent_main, handler, app, invoke.
# - Preferred signature if you can adapt:
#     async def run(input: str | None = None, files: list[str] | None = None, await_approval=None): ...
#
# Paste below:


In [ ]:
# Cell 3 — Generic runner: text/files input + approval handling
import os, io, uuid, asyncio, inspect, mimetypes, json
from pathlib import Path
from IPython.display import display, clear_output
import ipywidgets as W

UPLOAD_ROOT = Path(os.getenv('AGENTKIT_UPLOAD_ROOT', '.agentkit_uploads'))
UPLOAD_ROOT.mkdir(parents=True, exist_ok=True)

def _save_uploads(uploader):
    saved = []
    if not uploader.value:
        return saved
    session_dir = UPLOAD_ROOT / str(uuid.uuid4())
    session_dir.mkdir(parents=True, exist_ok=True)
    # ipywidgets returns dict keyed by filename (classic) or list of dicts (newer)
    items = uploader.value
    if isinstance(items, dict):
        iterable = items.values()
    else:
        iterable = items
    for item in iterable:
        name = item.get('name') or item.get('metadata', {}).get('name') or f'file_{len(saved)}'
        content = item.get('content') or item.get('data')
        path = session_dir / name
        with open(path, 'wb') as f:
            f.write(content)
        saved.append(str(path))
    return saved

async def await_approval(title='Approval required', details=None, approve_label='Approve', deny_label='Deny'):
    fut = asyncio.get_event_loop().create_future()
    header = W.HTML(f'<b>{title}</b>')
    body = W.HTML(f'<pre style="white-space:pre-wrap">{json.dumps(details, indent=2) if details is not None else ''}</pre>')
    approve_btn = W.Button(description=approve_label, button_style='success')
    deny_btn = W.Button(description=deny_label, button_style='danger')
    btns = W.HBox([approve_btn, deny_btn])
    box = W.VBox([header, body, btns])
    out = W.Output()
    display(box, out)

    def _resolve(val):
        if not fut.done():
            fut.set_result(val)
        box.close()
        out.close()

    approve_btn.on_click(lambda _: _resolve(True))
    deny_btn.on_click(lambda _: _resolve(False))
    return await fut

def _is_async_callable(fn):
    return inspect.iscoroutinefunction(fn) or inspect.isasyncgenfunction(fn)

def _build_kwargs(fn, text_input, file_paths):
    params = list(inspect.signature(fn).parameters.keys())
    kwargs = {}
    for name in params:
        lname = name.lower()
        if lname in ('input', 'text', 'prompt', 'message', 'query'):
            kwargs[name] = text_input
        elif lname in ('files', 'file_paths', 'filepaths', 'paths', 'attachments'):
            kwargs[name] = file_paths
        elif lname in ('await_approval', 'approval', 'approval_callback', 'confirm', 'approval_fn'):
            kwargs[name] = await_approval
    return kwargs

def _find_callable_by_name(name):
    ns = globals()
    obj = ns.get(name)
    if obj is None:
        # dotted access like my_app.run
        parts = name.split('.')
        obj = ns.get(parts[0])
        for part in parts[1:]:
            obj = getattr(obj, part, None)
            if obj is None:
                break
    if callable(obj):
        return obj
    # If it's an object with a .run()
    if obj is not None and hasattr(obj, 'run') and callable(getattr(obj, 'run')):
        return getattr(obj, 'run')
    return None

PREFERRED_NAMES = ['run', 'main', 'agent_main', 'invoke', 'handler', 'handle', 'app', 'agent', 'entrypoint']

def _discover_candidates():
    ns = globals()
    found = []
    for name in PREFERRED_NAMES:
        obj = ns.get(name)
        if callable(obj):
            found.append((name, obj))
        elif obj is not None and hasattr(obj, 'run') and callable(getattr(obj, 'run')):
            found.append((f'{name}.run', getattr(obj, 'run')))
    # De-duplicate by object id
    seen = set()
    uniq = []
    for name, obj in found:
        i = id(obj)
        if i in seen:
            continue
        seen.add(i)
        uniq.append((name, obj))
    return uniq

# UI
input_text = W.Textarea(
    value='',
    placeholder='Type your user input here (optional if only files).',
    description='Input:',
    layout=W.Layout(width='100%', height='120px')
)

files_upload = W.FileUpload(accept='', multiple=True, description='Attach files')

entry_dropdown = W.Dropdown(options=[], description='Entrypoint:', layout=W.Layout(width='50%'))
manual_name = W.Text(value='', placeholder='Or type a callable name like run or my_app.run', description='Manual:')

refresh_btn = W.Button(description='Refresh', icon='refresh')
run_btn = W.Button(description='Run', button_style='primary', icon='play')
output_area = W.Output()

def _refresh_candidates(_=None):
    cands = _discover_candidates()
    if not cands:
        entry_dropdown.options = []
        entry_dropdown.value = None
    else:
        entry_dropdown.options = [(name, obj) for name, obj in cands]
        entry_dropdown.value = entry_dropdown.options[0][1]

refresh_btn.on_click(_refresh_candidates)
_refresh_candidates()

def _pretty_display(result):
    with output_area:
        if result is None:
            print('✓ Completed (no return value).')
            return
        try:
            from openai import BaseModel
            # If it is a pydantic-like model
        except Exception:
            BaseModel = None
        try:
            import pandas as pd  # noqa
            if hasattr(result, 'to_dict'):
                print(json.dumps(result.to_dict(), indent=2, default=str))
                return
        except Exception:
            pass
        try:
            print(json.dumps(result, indent=2, default=str))
        except Exception:
            print(result)

async def _execute_selected():
    output_area.clear_output()
    file_paths = _save_uploads(files_upload)
    text_value = input_text.value or None

    fn = entry_dropdown.value
    if fn is None and manual_name.value.strip():
        fn = _find_callable_by_name(manual_name.value.strip())

    if fn is None:
        with output_area:
            print('No callable selected or found. Define a function like "run" in Cell 2 or type its name above.')
        return

    kwargs = _build_kwargs(fn, text_value, file_paths)

    try:
        if _is_async_callable(fn):
            result = await fn(**kwargs)
        else:
            result = fn(**kwargs)
            if inspect.isawaitable(result):
                result = await result
            elif inspect.isgenerator(result):
                with output_area:
                    print('Streaming results:')
                for chunk in result:
                    with output_area:
                        print(chunk)
                result = None
        _pretty_display(result)
    except Exception as e:
        with output_area:
            import traceback
            print('Execution failed:')
            print(''.join(traceback.format_exception(type(e), e, e.__traceback__)))

async def _run_wrapper(_=None):
    await _execute_selected()

def _on_run_clicked(_):
    asyncio.ensure_future(_run_wrapper())

run_btn.on_click(_on_run_clicked)

controls = W.VBox([
    input_text,
    files_upload,
    W.HBox([entry_dropdown, refresh_btn]),
    manual_name,
    run_btn,
])

display(controls, output_area)
